# SCF Workshop on databases and SQL

In [1]:
import requests
from pathlib import Path 
import time
import pandas as pd 
import sqlalchemy

In [3]:
db_filename = 'stackoverflow-2016.db' 
db_path = db_filename

In [4]:
engine = sqlalchemy.create_engine(f"sqlite:///{db_path}") 
connection = engine.connect()

In [5]:
engine.table_names()

/tmp/ipykernel_25/4162273999.py:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['answers', 'questions', 'questions_tags', 't1', 'table_example', 'users']

In [6]:
for exp in connection.execute('SELECT sql FROM sqlite_master;'):
    print(exp[0])

CREATE TABLE "questions" (
  "questionid" INTEGER,
  "creationdate" TEXT,
  "score" INTEGER,
  "viewcount" INTEGER,
  "title" TEXT,
  "ownerid" INTEGER
)
CREATE TABLE "answers" (
  "answerid" INTEGER,
  "questionid" INTEGER,
  "creationdate" TEXT,
  "score" INTEGER,
  "ownerid" INTEGER
)
CREATE TABLE "questions_tags" (
  "questionid" INTEGER,
  "tag" TEXT
)
CREATE TABLE "users" (
  "userid" INTEGER,
  "creationdate" TEXT,
  "lastaccessdate" TEXT,
  "location" TEXT,
  "reputation" INTEGER,
  "displayname" TEXT,
  "upvotes" INTEGER,
  "downvotes" INTEGER,
  "age" INTEGER,
  "accountid" INTEGER
)
CREATE VIEW question_contrasts AS
SELECT * FROM questions Q1 JOIN questions Q2
ON Q1.ownerid = Q2.ownerid
WHERE Q1.creationdate != Q2.creationdate
CREATE VIEW maxrep_step1 AS
SELECT *, MAX(reputation) AS maxRep
FROM answers A1
JOIN users U1
WHERE A1.ownerid = U1.userid
GROUP BY A1.questionid
LIMIT 10
CREATE VIEW maxrep_a AS
SELECT *, MAX(reputation) AS maxRep
FROM answers A1
JOIN users U1
WHERE A

## Self Joins

In [6]:
# Self join
query = '''
SELECT * 
FROM questions T1, questions T2
WHERE T1.ownerid = T2.ownerid
AND T1.creationdate != T2.creationdate
LIMIT 5
'''
question_contrasts = pd.read_sql(query,engine)
question_contrasts

,questionid,creationdate,score,viewcount,title,ownerid,questionid,creationdate,score,viewcount,title,ownerid
0,34552550,2016-01-01 00:00:03,0,108,Scope between methods,5684416,34558807,2016-01-01 18:27:40,0,44,Bug when displaying images in iOS app,5684416
1,34552550,2016-01-01 00:00:03,0,108,Scope between methods,5684416,34560451,2016-01-01 21:56:32,0,29,Array of rounded buttons,5684416
2,34552550,2016-01-01 00:00:03,0,108,Scope between methods,5684416,34572511,2016-01-03 01:34:23,0,36,App out of phone screen bounds,5684416
3,34552550,2016-01-01 00:00:03,0,108,Scope between methods,5684416,34573433,2016-01-03 04:48:47,0,54,Background color,5684416
4,34552550,2016-01-01 00:00:03,0,108,Scope between methods,5684416,34576840,2016-01-03 13:04:35,0,71,App not displaying properly on iphone6,5684416


In [7]:
# Remove duplicate pairs
query = '''
SELECT * 
FROM questions T1, questions T2
WHERE T1.ownerid = T2.ownerid
AND T1.creationdate < T2.creationdate
LIMIT 5
'''
question_contrasts = pd.read_sql(query,engine)
question_contrasts

,questionid,creationdate,score,viewcount,title,ownerid,questionid,creationdate,score,viewcount,title,ownerid
0,34552550,2016-01-01 00:00:03,0,108,Scope between methods,5684416,34558807,2016-01-01 18:27:40,0,44,Bug when displaying images in iOS app,5684416
1,34552550,2016-01-01 00:00:03,0,108,Scope between methods,5684416,34560451,2016-01-01 21:56:32,0,29,Array of rounded buttons,5684416
2,34552550,2016-01-01 00:00:03,0,108,Scope between methods,5684416,34572511,2016-01-03 01:34:23,0,36,App out of phone screen bounds,5684416
3,34552550,2016-01-01 00:00:03,0,108,Scope between methods,5684416,34573433,2016-01-03 04:48:47,0,54,Background color,5684416
4,34552550,2016-01-01 00:00:03,0,108,Scope between methods,5684416,34576840,2016-01-03 13:04:35,0,71,App not displaying properly on iphone6,5684416


## Set Operations

In [8]:
# Find all the questions about either R or Python or JAVA.
# Invesitage tag col
query = '''
SELECT DISTINCT tag
FROM questions_tags
'''
pd.read_sql(query,engine).size
query = '''
SELECT DISTINCT tag
FROM questions_tags
LIMIT 10
'''
pd.read_sql(query,engine)

,tag
0,c#
1,razor
2,flags
3,javascript
4,rxjs
5,node.js
6,audio
7,module
8,ffmpeg
9,php


In [79]:
# popular tags (has many views)
query = '''
SELECT DISTINCT tag
FROM questions_tags
WHERE questionid IN (SELECT questionid FROM questions ORDER BY viewcount)
LIMIT 20
'''
pd.read_sql(query,engine)

,tag
0,c#
1,razor
2,flags
3,javascript
4,rxjs
5,node.js
6,audio
7,module
8,ffmpeg
9,php


In [9]:
# Find all the questions about either R or Python (added Java)
query = '''
SELECT title 
FROM questions Q1, questions_tags T1
WHERE Q1.questionid = T1.questionid
AND T1.tag = 'r'
UNION
SELECT title 
FROM questions Q2, questions_tags T2
WHERE Q2.questionid = T2.questionid
AND T2.tag = 'python'
UNION
SELECT title 
FROM questions Q3, questions_tags T3
WHERE Q3.questionid = T3.questionid
AND T3.tag = 'java'
LIMIT 10
'''
pd.read_sql(query,engine)

,title
0,!arraylist.contains() not working?
1,!ls in Jupyter notebook (Python 3)
2,""" * "" vs "" : "" in R for modelling"
3,""" ImportError: cannot import name CaptchaField..."
4,""" No qualifying bean of type "" while DAO testi..."
5,""" could not find or load main class "" cmd, win..."
6,""" django.template.exceptions.TemplateSyntaxErr..."
7,""" javax.servlet.* "" and ""HttpServlet"" can't be..."
8,"""""""TypeError: only length-1 arrays can be conv..."
9,"""%matplotlib inline"" doesn't display unicode c..."


## String Processing

In [10]:
# Select the questions that have "java" but not "javascript" in their titles using regular expression syntax
query = '''
SELECT * 
FROM questions_tags
WHERE tag LIKE '%java'
LIMIT 10
'''
pd.read_sql(query,engine)

,questionid,tag
0,34553290,java
1,34553746,java
2,34554170,java
3,34554274,java
4,34555871,java
5,34556689,java
6,34556734,java
7,34557412,java
8,34557957,java
9,34560148,java


In [11]:
# calculate the length (in characters) of the title of each question
query = '''
SELECT title, LENGTH(title)
AS title_char_length
FROM questions
LIMIT 10
'''
pd.read_sql(query,engine)

,title,title_char_length
0,Scope between methods,21
1,Rails - Unknown Attribute - Unable to add a ne...,80
2,Selenium Firefox webdriver won't load a blank ...,85
3,Android Studio styles.xml Error,31
4,Java: reference to non-finial local variables ...,61
5,How to namespace routes in Rails 4 to set up F...,51
6,Force delete Session records for the current l...,69
7,Which data structure (full HABTM ?),35
8,"PHP: Phing, Phar, and phar.readonly",35
9,Is there a way to use flashlight on in Android...,76


In [12]:
# Process the creationdate field to create year, day, and month fields.
# with original
query = '''
SELECT questionid, ownerid, score, viewcount, creationdate,
    SUBSTRING(creationdate, 1,4) AS year,
    SUBSTRING(creationdate, 6,2) AS month,
    SUBSTRING(creationdate, 9,2) AS day 
FROM questions
LIMIT 5
'''
creationdate_orig = pd.read_sql(query,engine)
# without original
query = '''
SELECT questionid, ownerid, score, viewcount,
    SUBSTRING(creationdate, 1,4) AS year,
    SUBSTRING(creationdate, 6,2) AS month,
    SUBSTRING(creationdate, 9,2) AS day 
FROM questions
LIMIT 15
'''
creationdate_final = pd.read_sql(query,engine)

In [13]:
creationdate_orig

,questionid,ownerid,score,viewcount,creationdate,year,month,day
0,34552550,5684416,0,108,2016-01-01 00:00:03,2016,01,01
1,34552551,2457617,1,151,2016-01-01 00:00:07,2016,01,01
2,34552552,5732525,2,1942,2016-01-01 00:00:39,2016,01,01
3,34552554,5735112,0,153,2016-01-01 00:00:50,2016,01,01
4,34552555,4646288,-1,54,2016-01-01 00:00:51,2016,01,01


In [14]:
creationdate_final

,questionid,ownerid,score,viewcount,year,month,day
0,34552550,5684416.0,0,108,2016,01,01
1,34552551,2457617.0,1,151,2016,01,01
2,34552552,5732525.0,2,1942,2016,01,01
3,34552554,5735112.0,0,153,2016,01,01
4,34552555,4646288.0,-1,54,2016,01,01
5,34552557,506982.0,1,26,2016,01,01
6,34552558,NaN,1,190,2016,01,01
7,34552560,3540430.0,0,35,2016,01,01
8,34552563,4668.0,2,484,2016,01,01
9,34552565,4087330.0,2,308,2016,01,01


## Subqueries

In [15]:
# Subqueries
# Write a query that returns the title of each question and answer to each question from the user 
# with the highest reputation amongst all those answering the question.

query = '''
SELECT *
FROM questions Q 
JOIN (SELECT *, MAX(reputation) AS maxRep 
        FROM answers A JOIN users U 
        WHERE A.ownerid = U.userid 
        GROUP BY A.questionid) maxRepTable
WHERE Q.questionid = maxRepTable.questionid
LIMIT 10
'''
pd.read_sql(query,engine)

,questionid,creationdate,score,viewcount,title,ownerid,answerid,questionid,creationdate,score,...,creationdate:1,lastaccessdate,location,reputation,displayname,upvotes,downvotes,age,accountid,maxRep
0,34552550,2016-01-01 00:00:03,0,108,Scope between methods,5684416.0,34552570,34552550,2016-01-01 00:04:27,1,...,2008-09-30 02:31:01,2017-03-13 17:05:28,"Palo Alto, CA",73941,jtbandes,1334,402,24.0,12095,73941
1,34552551,2016-01-01 00:00:07,1,151,Rails - Unknown Attribute - Unable to add a ne...,2457617.0,34554360,34552551,2016-01-01 07:23:32,1,...,2014-05-06 07:44:36,2017-03-10 15:04:52,"Ahmadabad, India",5874,Gagan Gami,511,88,28.0,4429723,5874
2,34552552,2016-01-01 00:00:39,2,1942,Selenium Firefox webdriver won't load a blank ...,5732525.0,34555040,34552552,2016-01-01 09:39:33,2,...,2013-10-07 07:47:41,2017-03-10 10:43:37,None,96,Hubert,4,0,NaN,3401741,96
3,34552554,2016-01-01 00:00:50,0,153,Android Studio styles.xml Error,5735112.0,34552709,34552554,2016-01-01 00:36:05,0,...,2015-10-13 18:23:16,2017-03-13 18:45:33,None,1275,miqdadamirali,179,45,NaN,7115774,1275
4,34552555,2016-01-01 00:00:51,-1,54,Java: reference to non-finial local variables ...,4646288.0,34552652,34552555,2016-01-01 00:21:38,1,...,2008-09-18 19:36:30,2017-03-13 21:40:57,"Portland, OR",60410,Jim Garrison,3018,12050,NaN,9760,60410
5,34552557,2016-01-01 00:01:12,1,26,How to namespace routes in Rails 4 to set up F...,506982.0,34585886,34552557,2016-01-04 06:38:01,1,...,2010-11-13 21:10:13,2017-03-04 08:04:03,None,112,Wemmick,95,0,NaN,238236,112
6,34552558,2016-01-01 00:01:27,1,190,Force delete Session records for the current l...,NaN,34552958,34552558,2016-01-01 01:40:56,0,...,2010-02-15 13:10:04,2017-03-13 16:01:53,India,2959,Pankaj,931,532,36.0,375765,2959
7,34552560,2016-01-01 00:01:50,0,35,Which data structure (full HABTM ?),3540430.0,34552698,34552560,2016-01-01 00:33:17,0,...,2014-04-16 09:02:57,2016-07-24 12:40:33,None,11,user3540430,0,0,NaN,4335954,11
8,34552563,2016-01-01 00:02:22,2,484,"PHP: Phing, Phar, and phar.readonly",4668.0,35120728,34552563,2016-01-31 23:16:26,3,...,2016-01-31 23:11:43,2016-03-19 16:38:32,None,31,AndovaBegarin,0,0,NaN,7748745,31
9,34552565,2016-01-01 00:02:53,2,308,Is there a way to use flashlight on in Android...,4087330.0,34554256,34552565,2016-01-01 07:04:21,1,...,2009-10-19 11:02:23,2017-03-13 16:05:11,None,26973,Alex Cohn,2947,50,NaN,65347,26973


In [16]:
# Write a query that would return the users who have asked a question with the Python tag.
query = '''
SELECT displayname, userid
FROM users
WHERE userid IN (SELECT ownerid 
                    FROM questions 
                    JOIN questions_tags
                    ON questions.questionid = questions_tags.questionid
                    WHERE tag = 'python')
LIMIT 10
'''
pd.read_sql(query,engine)

,displayname,userid
0,doekman,56
1,ESV,150
2,Anthony Main,258
3,gyurisc,260
4,dlamblin,459
5,Rob Lachlan,688
6,Andrew,826
7,Ryan Doherty,956
8,Rob Hunter,1145
9,Grey Panther,1265


## Additional challenge questions

In [15]:
# Create a frequency list of the tags used in the top 100 most answered questions
query = '''
SELECT T.tag, COUNT(*) AS tagCount
FROM questions_tags T 
WHERE questionid IN (SELECT questionid
                    FROM answers
                    GROUP BY questionid
                    ORDER BY COUNT(*) DESC
                    LIMIT 100) 
GROUP BY T.tag 
ORDER BY tagCount DESC
LIMIT 15
'''
pd.read_sql(query,engine)

,tag,tagCount
0,android,11
1,java,6
2,angular2,6
3,python,5
4,android-studio,5
5,typescript,4
6,ios10,4
7,jquery,3
8,javascript,3
9,ios,3


In [36]:
# How would you find all the answers associated with the user with the most upvotes?

# Step 1: Find user with most upvotes
query = '''
SELECT userid FROM (SELECT MAX(upvotes), userid FROM users)
'''
most_upvotes = pd.read_sql(query,engine)
most_upvotes

,userid
0,998692


In [42]:
# Step 1: Find answers assoc with user with most upvotes
query = '''
SELECT answerid, ownerid
FROM answers
WHERE ownerid 
IN (SELECT userid FROM (SELECT MAX(upvotes), userid FROM users)) 
LIMIT 20
'''
pd.read_sql(query,engine)

,answerid,ownerid
0,37423148,998692


## Window functions

In [49]:
# Use a window function to compute the average viewcount for each ownerid 
# for the 10 questions preceding each question.
query = '''
SELECT *, AVG(viewcount) 
    OVER (PARTITION BY ownerid
        ORDER BY JULIANDAY(creationdate)
        ROWS BETWEEN 10 PRECEDING AND 1 PRECEDING) 
    AS avg_view           
    FROM questions 
    WHERE ownerid IS NOT null 
    LIMIT 30
'''
pd.read_sql(query,engine)

,questionid,creationdate,score,viewcount,title,ownerid,avg_view
0,41114667,2016-12-13 06:09:50,0,55,How to implement autovivification for Ruby str...,13,NaN
1,35473568,2016-02-18 05:31:01,0,38,Postgres DB Password not being set in Kubernet...,25,NaN
2,36177186,2016-03-23 11:39:08,0,213,PDFsharp on Azure with Custom Fonts,33,NaN
3,38792937,2016-08-05 15:32:30,0,803,How do you fake your location with the google ...,33,213.0
4,39178864,2016-08-27 08:01:24,-2,25,How can you pipe email to c# ASP.net web site ...,33,508.0
5,39958722,2016-10-10 12:50:36,4,51,How can I work out time difference between git...,33,347.0
6,37158688,2016-05-11 09:40:11,1,38,How to make my PyObjC application AppleScript-...,56,NaN
7,37211626,2016-05-13 13:44:03,2,72,What does «event coreslct» do in AppleScript,56,38.0
8,39492672,2016-09-14 14:13:19,0,29,How can I use `psql` without installing via brew,56,55.0
9,37732377,2016-06-09 17:16:10,0,26,Blurry/jagged text on iOS,62,NaN


In [70]:
# Question Find the users who have asked one question that is highly-viewed (viewcount > 1000) 
# with their remaining questions not highly-viewed (viewcount < 20).

# step 1 : find the viewcount, rank of each question, number of questions asked by the user (maxcount)
query = '''
SELECT *, 
RANK() OVER w AS rank, 
MAX(viewcount) OVER w 
    AS maxcount FROM questions 
    WHERE ownerid IS NOT null
    WINDOW w AS (PARTITION BY ownerid ORDER BY viewcount DESC)
ORDER BY viewcount DESC
LIMIT 15
'''
pd.read_sql(query,engine)

,questionid,creationdate,score,viewcount,title,ownerid,rank,maxcount
0,36668374,2016-04-16 18:57:19,20,196469,"How to solve ""server DNS address could not be ...",1707976,1,196469
1,37806538,2016-06-14 08:16:21,223,174790,Code signing is required for product type 'App...,1554347,1,174790
2,34814368,2016-01-15 15:24:36,206,134399,"""Gradle Version 2.10 is required."" Error",3319176,1,134399
3,35890257,2016-03-09 11:25:05,51,129874,Android- Error:Execution failed for task ':app...,1118886,1,129874
4,34579099,2016-01-03 16:55:16,8,129624,Fatal error: Uncaught Error: Call to undefined...,3656666,1,129624
5,35990995,2016-03-14 15:01:17,104,127764,Unsupported major.minor version 52.0 in my app,1629278,1,127764
6,35588699,2016-02-23 21:37:06,57,126752,Response to preflight request doesn't pass acc...,2896963,1,126752
7,35062852,2016-01-28 13:28:39,730,112000,NPM vs. Bower vs. Browserify vs. Gulp vs. Grun...,2761509,1,112000
8,37937984,2016-06-21 07:23:00,202,109422,Git refusing to merge unrelated histories,2670370,1,109422
9,37280274,2016-05-17 15:21:49,23,106995,"""SyntaxError: Unexpected token < in JSON at po...",4043633,1,106995


In [72]:
# use step 1 as subquery and filter on the conditions: (rank = 2), (viewcount > 1000), and (viewcount < 20)
query = '''
SELECT * FROM (SELECT *, 
    RANK() OVER w as RANK,
    MAX(viewcount) OVER w AS maxcount
    from questions where ownerid is not null
    window w as (partition by ownerid order by viewcount desc))
where rank = 2 
and viewcount < 100 
and maxcount > 1000
limit 20
'''
pd.read_sql(query,engine)

,questionid,creationdate,score,viewcount,title,ownerid,rank,maxcount
0,36033340,2016-03-16 10:43:27,-2,30,Cython Build Service,194,2,2771
1,40534958,2016-11-10 18:50:26,3,70,Wildcard file include in .NET Core project wit...,767,2,2978
2,38987128,2016-08-17 02:18:33,0,91,How to git log of all branches of a remote?,1630,2,1092
3,39045133,2016-08-19 17:56:49,0,26,Is it possible to use ttml or vtt with the htm...,3537,2,1581
4,35265225,2016-02-08 08:32:51,0,89,Rails Counter Cache On the same model?,4038,2,2325
5,40719406,2016-11-21 11:46:31,1,51,BDD: Should I add multiple givens and outcomes...,4830,2,3755
6,37211298,2016-05-13 13:27:35,2,49,Accessing an array as a struct vs undefined be...,6345,2,2670
7,35168135,2016-02-03 03:47:45,0,38,"Do I have to hard-code / localise myself the ""...",6832,2,1201
8,38173710,2016-07-03 19:41:45,0,93,Amazon S3: Can clients see the file before upl...,8446,2,3605
9,38644194,2016-07-28 18:31:40,0,37,"NodeJS spawned process works on Mac, fails on ...",8513,2,1282
